binary coded genetic algorithm using deap library

In [1]:
import os
import numpy as np
import random
from deap import base, creator, tools, algorithms
import csv

# === Folder Path ===
GAP_FOLDER = r"C:\Users\MD KAIF\Desktop\lab_assignments\ec_lab"

# === GA Parameters ===
POP_SIZE = 100
GENS = 200
CX_PROB = 0.9
MUT_PROB = 0.05
TOURN_SIZE = 3

# === GAP Parser ===
def parse_gap_file(filepath):
    with open(filepath, 'r') as f:
        lines = [line.strip() for line in f if line.strip()]
    index = 0
    num_instances = int(lines[index])
    index += 1
    instances = []

    for _ in range(num_instances):
        m, n = map(int, lines[index].split())
        index += 1

        util, res = [], []
        for _ in range(m):
            row = []
            while len(row) < n:
                row += list(map(int, lines[index].split()))
                index += 1
            util.append(row)
        for _ in range(m):
            row = []
            while len(row) < n:
                row += list(map(int, lines[index].split()))
                index += 1
            res.append(row)
        cap = []
        while len(cap) < m:
            cap += list(map(int, lines[index].split()))
            index += 1

        instances.append((m, n, np.array(util), np.array(res), np.array(cap)))
    return instances

# === Decode + Repair ===
def decode(ind, n_users, m_servers):
    bpg = int(np.ceil(np.log2(m_servers)))
    alloc = []
    for i in range(n_users):
        bits = ind[i * bpg:(i + 1) * bpg]
        val = int("".join(map(str, bits)), 2)
        alloc.append(val % m_servers)
    return alloc

def repair_allocation(alloc, res, cap):
    m, n = res.shape
    usage = np.zeros(m)
    for u, s in enumerate(alloc):
        usage[s] += res[s][u]
    for s in range(m):
        if usage[s] > cap[s]:
            for u in range(n):
                if alloc[u] == s:
                    for new_s in range(m):
                        if new_s != s and usage[new_s] + res[new_s][u] <= cap[new_s]:
                            usage[s] -= res[s][u]
                            usage[new_s] += res[new_s][u]
                            alloc[u] = new_s
                            break
                    if usage[s] <= cap[s]:
                        break
    return alloc, usage

# === Fitness ===
def evaluate(ind, m, n, util, res, cap):
    alloc = decode(ind, n, m)
    alloc, usage = repair_allocation(alloc, res, cap)
    for s in range(m):
        if usage[s] > cap[s]:
            return 0,
    total = sum(util[alloc[u]][u] for u in range(n))
    return total,

# === Binary GA Solver ===
def solve_gap_instance(m, n, util, res, cap):
    bpg = int(np.ceil(np.log2(m)))
    chrom_len = bpg * n

    if "FitnessMax" not in creator.__dict__:
        creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    if "Individual" not in creator.__dict__:
        creator.create("Individual", list, fitness=creator.FitnessMax)

    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, chrom_len)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("mate", tools.cxOnePoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.01)
    toolbox.register("select", tools.selTournament, tournsize=TOURN_SIZE)
    toolbox.register("evaluate", evaluate, m=m, n=n, util=util, res=res, cap=cap)

    pop = toolbox.population(n=POP_SIZE)
    algorithms.eaSimple(pop, toolbox, cxpb=CX_PROB, mutpb=MUT_PROB, ngen=GENS, verbose=False)
    best = tools.selBest(pop, k=1)[0]

    best_alloc = decode(best, n, m)
    best_alloc, usage = repair_allocation(best_alloc, res, cap)
    total = sum(util[best_alloc[u]][u] for u in range(n))
    return best_alloc, total

# === Main Execution ===
binary_ga_results = []

def sort_key(filename):
    return int(''.join(filter(str.isdigit, filename)))

for file in sorted(os.listdir(GAP_FOLDER), key=sort_key):
    if file.endswith(".txt"):
        filepath = os.path.join(GAP_FOLDER, file)
        instances = parse_gap_file(filepath)

        for idx, (m, n, util, res, cap) in enumerate(instances):
            try:
                best_alloc, total = solve_gap_instance(m, n, util, res, cap)
                print(f"Processing Instance {idx + 1} from {file}")
                print(f"Task Assignments: {best_alloc}")
                print(f"Total Binary GA Utility: {float(total)}\n")

                binary_ga_results.append((file, idx + 1, best_alloc, float(total)))
            except Exception as e:
                print(f"⚠ Error in instance {idx + 1} from {file}: {e}\n")

# === Save Binary GA Results to CSV ===
output_csv_path = os.path.join(r"C:\Users\MD KAIF\Desktop\assignment3_soln", "binary_ga_results.csv")

with open(output_csv_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["File", "Instance", "Task Assignments", "Total Utility"])
    for result in binary_ga_results:
        file_name, instance_no, task_assignments, total_utility = result
        writer.writerow([file_name, instance_no, task_assignments, total_utility])

print(f"\n✅ Binary GA results saved to: {output_csv_path}")


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (2807035161.py, line 141)